In [66]:
import numpy as np
np.random.seed(0)

#Activation functions

def sin_function(x):
    return np.sin(x)

def relu(x):
    return np.maximum(0, x)

def tanh(x):
    return np.tanh(x)

#Function to approximate
def f(x):
    return np.sin(3*np.pi*x + 3*np.pi/20) * np.cos(2*np.pi*x + np.pi/10) + 2


In [67]:
#Function to build the mass matrix - M
from scipy.integrate import quad

def mass_matrix(func,sigma, weights, biases, n):
    M = np.zeros([n, n])
    
    for i in range(0, n):
        for j in range(0, n):
            M[i, j] = quad(func, 0, 1, args=(weights, biases,i+1,j+1,sigma), limit=1000)[0]

    return M

In [68]:
#Single basis function - Neural Network
def single_neural_net(x,weights,biases,i, sigma): 

    return sigma(weights[i]*x+biases[i])


In [69]:
#Basis function multiplication helper for quadrature
def double_neural_net(x,weights,biases,i,j, sigma): 

    return single_neural_net(x,weights,biases,i,sigma)*single_neural_net(x,weights,biases,j,sigma)

In [70]:
#Bias helper function for quadrature
def b_neural_net(x, weights, biases,i, sigma,f):
    return single_neural_net(x,weights,biases,i, sigma)*f(x)


In [71]:
#quadrature for the bias matrix - B
def b_matrix(func,sigma, weights, biases,f, neurons):
    b = np.zeros([neurons, 1])
    for i in range(0, neurons):
            b[i] = quad(func, 0, 1, args=(weights, biases,i,sigma,f), limit=1000)[0]

    return b

In [72]:
#Function to build the approximation vector
def u_nn(x, weights, biases, sigma,collocation, neurons, a):
    u = np.zeros(collocation)
    
    for j in range(0,collocation):
        for i in range(0,neurons):
            
            u[j] += single_neural_net(x[j], weights, biases, i, sigma)*a[i]
        
    return u


In [73]:
#helper function for eigenvalues and eigenvectors
from scipy.linalg import eig
def get_eig(M):
    D,V = eig(M, left=False, right=True)


    idx = D.argsort()[::-1]   
    D = D[idx]
    V = V[:,idx]

    D = np.flip(D)
    V = np.fliplr(V)

    return D,V

In [74]:
#Function to calculate L2 loss
def l2_loss(f_x, u_nn_result):
    return np.linalg.norm(f_x - u_nn_result)**2

In [75]:
#Function to build the Phi matrix, made up of n basis functions
def phi_matrix(x,weights,biases,sigma,neurons,collocation):
    phi = np.zeros((collocation,neurons))
    for i in range(0,neurons):
        for j in range(0,collocation):

            phi[j,i] = sigma(weights[i]*x[j]+biases[i])
    return phi


In [76]:
def evaluate_func_approx(x,f, weights, biases, sigma, neurons, collocation,verbose=False):
    phi = phi_matrix(x,weights,biases,sigma,neurons,collocation)
    f_func = f(x)
    a = np.linalg.pinv(phi)@f_func

    u = u_nn(x, weights, biases, sigma, collocation,neurons, a)

    loss = l2_loss(f_func, u)
    if verbose:
        print("="*20)
        print("Sigma:", sigma.__name__)
        print("Number of collocation points:", collocation)
        print("Number of neurons:", neurons)
        print("L2 Loss:", loss)
        print("="*20)
    return loss

In [77]:
import matplotlib.pyplot as plt

def compute_loss_values(neurons, collocations, sigmas, Rs,xmin=0,xmax=1,verbose=False):
    loss_values = []
    #np.random.seed(0)  # Set a seed for reproducibility
    for sigma in sigmas:
        for neuron in neurons:
            for collocation in collocations:
                for R in Rs:
                    x = np.linspace(xmin, xmax, collocation)
                    weights = np.random.uniform(-R, R, neuron)
                    biases = np.random.uniform(-R, R, neuron)
                    loss = evaluate_func_approx(x, f, weights, biases, sigma, neuron, collocation, verbose=verbose)
                    loss_values.append((loss, collocation, neuron, sigma.__name__,R))  # Add activation function name
    return loss_values

In [78]:
def print_loss_by_collocation(loss_values):
    """
    Prints the loss values grouped by collocation point, activation function, and range R.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each collocation point
    collocation_loss_data = {collocation: [] for _, collocation, _, _, _ in loss_values}

    # Populate the dictionary with (neuron, loss, activation_func_name, R) tuples for each collocation point
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        collocation_loss_data[collocation].append((neuron, loss, activation_func_name, R))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the collocation points and print the relevant data
        for collocation, data in collocation_loss_data.items():
            print(f"# of Collocation points: {collocation}")
            
            # Print the neurons, loss values, and R for the current activation function
            for neuron, loss, act_func_name, R in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Neurons: {neuron}, Loss: {loss:.10e}, R: {R}")



In [79]:
def print_loss_by_neuron(loss_values):
    """
    Prints the loss values grouped by neuron, activation function, and range R.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each neuron
    neuron_loss_data = {neuron: [] for _, _, neuron, _, _ in loss_values}

    # Populate the dictionary with (collocation, loss, activation_func_name, R) tuples for each neuron
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        neuron_loss_data[neuron].append((collocation, loss, activation_func_name, R))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the neurons and print the relevant data
        for neuron, data in neuron_loss_data.items():
            print(f"# of Neurons: {neuron}")
            
            # Print the collocation points, loss values, and R for the current activation function
            for collocation, loss, act_func_name, R in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Collocation points: {collocation}, Loss: {loss:.10e}, R: {R}")




In [80]:
def print_loss_by_R(loss_values):
    """
    Prints the loss values grouped by the range R, activation function, collocation points, and neurons.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each R value
    R_loss_data = {R: [] for _, _, _, _, R in loss_values}

    # Populate the dictionary with (collocation, neuron, loss, activation_func_name) tuples for each R value
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        R_loss_data[R].append((collocation, neuron, loss, activation_func_name))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the R values and print the relevant data
        for R, data in R_loss_data.items():
            print(f"Range R: {R}")
            
            # Print the collocation points, neurons, and loss values for the current activation function
            for collocation, neuron, loss, act_func_name in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Collocation points: {collocation}, # of Neurons: {neuron}, Loss: {loss:.10e}")

In [81]:
collocation = 100
neurons = 50
sigma = sin_function
x = np.linspace(0,1,collocation)
weights = np.random.uniform(-1,1,neurons)
biases = np.random.uniform(-1,1,neurons)
test = evaluate_func_approx(x,f, weights, biases, sigma, neurons, collocation,verbose=True)

Sigma: sin_function
Number of collocation points: 100
Number of neurons: 50
L2 Loss: 0.08512083411934636


# Test 1 - Increasing #of neurons
Keeping collocations fixed at 50, and using ReLU, tanh, and sin as activation functions

In [82]:
neurons = [5,10,20,30,40,50,100]
sigmas = [relu,tanh,sin_function]
collocations = [50]
Rs = [1]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,verbose=False)
#print_loss_by_R(loss_values)
#loss_values

# Test 2 - Increasing Collocation points
Keeping neurons fixed at 20, and the same activation functions

In [83]:
neurons = [20]
sigmas = [tanh,relu,sin_function]
collocations = [10,50,100,200,500]
Rs = [1]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,verbose=False)

In [84]:
#print_loss_by_neuron(loss_values)

# Test 3 - Increasing the sampling range R
Keeping neurons and collocations fixed, increasing R to increase chance that w_i and b_i are further away from w_j and b_j

In [85]:
neurons = [30]
sigmas = [tanh,relu,sin_function]
collocations = [50]
Rs = [1,2,4,5,10]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,verbose=False)

In [86]:
#print_loss_by_collocation(loss_values)

In [87]:
neurons = [10,20,30,50,100]
sigmas = [tanh,relu,sin_function]
collocations = [5,10,30,50,100]
Rs = [1,2,4,5,10]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,verbose=False)

In [88]:
#print_loss_by_collocation(loss_values)

# Building Mass matrix

In [89]:
from scipy.integrate import quad
from numpy.linalg import cond
from scipy.linalg import eig

def mass_matrix(func,sigma, weights, biases, neurons):
    M = np.zeros([neurons, neurons])
    
    for i in range(0, neurons):
        for j in range(0, neurons):
            M[i, j] = quad(func, 0, 1, args=(weights, biases,i,j,sigma), limit=1000)[0]

    return M

In [171]:
neurons = 20
R=10
collocation = 10
x = np.linspace(0,1,collocation)
weights = np.random.uniform(-R,R,neurons)
biases = np.random.uniform(-R,R,neurons)
sigma = sin_function
M = mass_matrix(double_neural_net,sigma, weights, biases, neurons)
M

array([[ 5.03629233e-01, -2.03816594e-01, -3.56864545e-01,
         1.99093072e-01,  2.61573437e-01,  3.46475774e-02,
         4.14404076e-01, -3.17610211e-01, -4.78769183e-02,
        -1.02233116e-01, -4.27505846e-01,  1.54976221e-01,
        -2.88827610e-01,  1.89903947e-01,  3.33870699e-01,
         1.86772591e-01, -3.08547512e-01, -4.90171689e-02,
         1.23214863e-01, -4.23509637e-01],
       [-2.03816594e-01,  4.05264567e-01,  3.09535112e-01,
         6.43438507e-02,  3.58463897e-02, -2.89367686e-02,
        -2.60687488e-01, -1.50355219e-01, -1.12226779e-01,
        -1.86464235e-01, -1.78610892e-02, -2.13823067e-01,
         9.85981776e-02, -3.54340588e-01, -3.77933731e-01,
        -2.41244454e-02,  1.28263651e-01,  3.58881489e-01,
        -2.63960405e-01,  3.13495417e-01],
       [-3.56864545e-01,  3.09535112e-01,  5.69907119e-01,
         1.06547375e-01,  3.38108497e-02, -8.56365887e-02,
        -6.34109998e-01, -8.85802379e-02,  1.41683173e-02,
        -3.04206515e-01,  3.6

In [145]:
det_m = np.linalg.det(M)
det_m

2.1985259371182663e-52

In [172]:
cond_m = np.linalg.cond(M)
cond_m

2.7010446547906883e+17

In [155]:
b = b_matrix(b_neural_net,sigma,weights, biases,f,neurons)
#b

In [156]:
a = np.linalg.solve(M,b)
#a

In [157]:
u = u_nn(x,weights,biases,sigma,collocation,neurons,a)

/tmp/ipykernel_11828/1981966750.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  u[j] += single_neural_net(x[j], weights, biases, i, sigma)*a[i]


In [158]:
u

array([2.4643962 , 2.53972107, 1.91512868, 2.34547873, 2.99187966,
       2.43252066, 1.90459509, 2.46927764, 2.50180138, 1.55217831])

In [159]:
loss = l2_loss(f(x), u)
print("L2 Loss:", loss)

L2 Loss: 0.0016409959781178282
